In [ ]:
from allennlp.predictors.predictor import Predictor
from textblob import TextBlob

predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
text = "Alice met Bob. He was angry at her."
coref_resolved = predictor.predict(document=text)["document"]

sentiment = TextBlob(" ".join(coref_resolved)).sentiment
print(sentiment)  # Sentiment(polarity=-0.5, subjectivity=0.6)